<a href="https://colab.research.google.com/github/joselopez11c/mi-primer-proyecto/blob/main/remates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pdfplumber
!pip install curlify
nro_remate = '14045'
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive
import os
import pdfplumber
import tempfile
import curlify

In [39]:
url = 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml'
urlLogin = 'https://remaju.pj.gob.pe/remaju/pages/seguridad/login.xhtml'

headers = {
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,es;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Faces-Request': 'partial/ajax',
    'Origin': 'https://remaju.pj.gob.pe',
    'Referer': 'https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

response = requests.get(urlLogin, headers=headers)
cookie_header = 'JSESSIONID=' + response.cookies.get_dict().get('JSESSIONID')
headers['Cookie'] = cookie_header

def fetch_page(first_row):
  dataPagination = {
      'javax.faces.partial.ajax': 'true',
      'javax.faces.source': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.execute': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.partial.render': 'formBuscarRemateExterno:listaRemate',
      'javax.faces.behavior.event': 'page',
      'javax.faces.partial.event': 'page',
      'formBuscarRemateExterno:listaRemate_pagination': 'true',
      'formBuscarRemateExterno:listaRemate_skipChildren': 'true',
      'formBuscarRemateExterno:listaRemate_first': str(first_row),
      'formBuscarRemateExterno:listaRemate_rows': '12',
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
  }
  response = requests.post(url, headers=headers, data=dataPagination)
  return response

def downloadFile(idFile):
  headers = {
      'Accept': 'application/xml, text/xml, */*; q=0.01',
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': cookie_header
  }

  dataFile = {
      'formBuscarRemateExterno': 'formBuscarRemateExterno',
      idFile: '',
      'javax.faces.ViewState': view_state_value
  }

  response = requests.post(url, headers=headers, data=dataFile)
  return response


page = fetch_page(3) ####################################################### LLAMADA A LA PAGINA

soup = BeautifulSoup(page.content, 'xml')

update_tag = soup.find('update', {'id': 'j_id1:javax.faces.ViewState:0'})
view_state_value = update_tag.text

cdata_content = soup.find('update').text

# Ahora parseamos el contenido del CDATA, que es HTML
soup_html = BeautifulSoup(cdata_content, 'html.parser')

# Buscar todos los div con la clase 'card azul'
cards = soup_html.find_all('div', class_='card azul')

data = []
for card in cards:
    # Obtener número de remate y convocatoria
    remate_info = card.find('span', class_='text-bold label-danger h6').get_text(strip=True) if card.find('span', class_='text-bold label-danger h6') else 'No disponible'

    # Dividir la información de remate en número y convocatoria
    if remate_info != 'No disponible':
        parts = remate_info.split(" - ")
        remate_number = parts[0] if len(parts) > 0 else 'No disponible'
        convocatoria = parts[1] if len(parts) > 1 else 'No disponible'
    else:
        remate_number = 'No disponible'
        convocatoria = 'No disponible'

    # Ubicación, buscando la etiqueta específica y extrayendo el texto siguiente
    location_tag = card.find('i', class_='fa fa-map-marker')
    location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'

    # Descripción del inmueble, buscando el contenedor específico
    description = card.find('div', class_='texto-info-scroll').get_text(strip=True) if card.find('div', class_='texto-info-scroll') else 'No disponible'

    # Buscar todos los scripts dentro del card
    scripts = card.find_all('script')
    script_id = scripts[-1].get('id')[:-2] if scripts and scripts[-1].get('id').endswith('_s') else 'Script ID no encontrado'

    # Guardar los datos extraídos en la lista
    data.append({
        'Numer': remate_number,
       # 'Convocation': convocatoria,
       # 'Location': location,
       # 'Description': description,
        'Script ID': script_id,
       # 'View Code' : view_state_value,
       # 'Cookie header' : cookie_header
    })

# Crear un DataFrame con los datos recolectados
df = pd.DataFrame(data)

df['Banco_Creditor'] = ""
df['Expediente_Judicial'] = ""
df['Juzgado'] = ""

for index, row in df.iterrows():
    file_name = row['Numer'] + '.pdf'
    response = downloadFile(row['Script ID'])  # Asegúrate de que esta función esté correctamente definida

    if response.ok and 'application/pdf' in response.headers.get('Content-Type', ''):
        pdf_path = 'downloaded_file.pdf'
        with open(pdf_path, 'wb') as f:
            f.write(response.content)

        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
              text += page.extract_text().replace('\n', ' ') if page.extract_text() else ''

        # Usar expresiones regulares para extraer la información
        #print(file_name + ': ' + text)
        banco_creditor = re.search(r"seguidos por (.*?),", text, re.DOTALL)
        expediente = re.search(r"Expediente Judicial (.*?),", text, re.DOTALL)
        juzgado = re.search(r"tramitado ante el (.*?),", text, re.DOTALL)

        df.at[index, 'Banco_Creditor'] = banco_creditor.group(1) if banco_creditor else "No encontrado"
        df.at[index, 'Expediente_Judicial'] = expediente.group(1) if expediente else "No encontrado"
        df.at[index, 'Juzgado'] = juzgado.group(1).replace('\n', ' ').strip() if juzgado else "No encontrado"

        os.remove(pdf_path)

    else:
        print("Failed to download a valid PDF file.")

df

<ipython-input-39-3c627b6621d5>:89: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  location = location_tag.find_next_sibling(text=True).strip() if location_tag else 'No disponible'


,Numer,Script ID,Banco_Creditor,Expediente_Judicial,Juzgado
0,Remate N° 14063,formBuscarRemateExterno:listaRemate:3:j_idt164,CREDICORP CAPITAL SOCIEDAD TITULIZADORA S.A.,N° 00778-2011-0-2701-JM-CI-01,JUZGADO CIVIL - SEDE TAMBOPATA DE MADRE DE DIOS
1,Remate N° 14062,formBuscarRemateExterno:listaRemate:4:j_idt164,BANCO DE CREDITO DEL PERU,N° 00782-2021-0-2001-JR-CI- 03,3° JUZG. CIVIL -Av.F.Chirichigno 351-Urb.El Ch...
2,Remate N° 14061,formBuscarRemateExterno:listaRemate:5:j_idt164,CAJA MUNIC.DE AHORR.Y CRED.DE TACNA S.A.,N° 00269-2022-0-2801-JP-CI-03,Juzgado de Paz Letrado Mixto de Mariscal Nieto...
3,Remate N° 14060,formBuscarRemateExterno:listaRemate:6:j_idt164,CMAC PIURA S.A.C.,N° 00051-2019-0-2001-JR-CI-03,3° JUZG. CIVIL -Av.F.Chirichigno 351-Urb.El Ch...
4,Remate N° 14059,formBuscarRemateExterno:listaRemate:7:j_idt164,CAJA MUNICIPAL DE AHORRO Y CREDITO DE AREQUIPA...,N° 00032-2022-0-0404-JP-CI-01,JUZGADO DE PAZ LETRADO DE APLAO DE AREQUIPA
5,Remate N° 14058,formBuscarRemateExterno:listaRemate:8:j_idt164,BANCO BBVA PERU,N° 09879-2023-0-1817-JR-CO-05,5°JUZGADO CIVIL-COMERCIAL DE LIMA
6,Remate N° 14057,formBuscarRemateExterno:listaRemate:9:j_idt164,RICARDINA TAGAMI DE CHANG,N° 00088-2006-0-1301-JR-CI-01,1° JUZGADO CIVIL - Sede Barranca DE HUAURA
7,Remate N° 14056,formBuscarRemateExterno:listaRemate:10:j_idt164,JOSE JESUS BUGOSEN CHALUJA,N° 08569-2018-0-1817-JR-CO-16,16°JUZGADO CIVIL-COMERCIAL DE LIMA
8,Remate N° 14055,formBuscarRemateExterno:listaRemate:11:j_idt164,JOSE JESUS BUGOSEN CHALUJA,N° 08569-2018-0-1817-JR-CO-16,16°JUZGADO CIVIL-COMERCIAL DE LIMA
9,Remate N° 14054,formBuscarRemateExterno:listaRemate:12:j_idt164,ALFONSO FLORES CRUZ,N° 00740-2022-0-2301-JR-CI-04,4º JUZGADO CIVIL (EX J. CIVIL T. GREGORIO A.) ...


In [5]:
import requests

url = "https://remaju.pj.gob.pe/remaju/pages/publico/remateExterno.xhtml"

headers = {
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': cookie_header
}

data = {
    'formBuscarRemateExterno': 'formBuscarRemateExterno',
    'formBuscarRemateExterno:listaRemate:0:j_idt171': '',
    'javax.faces.ViewState': view_state_value
}

print(headers)
print(data)

response = requests.post(url, headers=headers, data=data)

print(response)


# Check if the response contains content
if response.ok and 'application/pdf' in response.headers.get('Content-Type', ''):
    # Saving the PDF to a local file
    pdf_path = 'downloaded_file.pdf'
    with open(pdf_path, 'wb') as f:
        f.write(response.content)

    # Reading the PDF using pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ''  # Append extracted text, use '' if None is returned

    # Print the extracted text
    print(text)

    # Optionally, remove the PDF file if not needed
    os.remove(pdf_path)

else:
    print("Failed to download a valid PDF file.")


df

{'Accept': 'application/xml, text/xml, */*; q=0.01', 'Content-Type': 'application/x-www-form-urlencoded', 'Cookie': 'JSESSIONID="tTVzWVs8aA0lJ53sHXPS3qUohGzP8KnpK7n8xaqz.P-Nodo5:SRemaju7-3"'}
{'formBuscarRemateExterno': 'formBuscarRemateExterno', 'formBuscarRemateExterno:listaRemate:0:j_idt171': '', 'javax.faces.ViewState': '1332934851288525934:8043642494852451096'}
<Response [200]>
Failed to download a valid PDF file.


In [33]:
import re

text = """
Remate N° 14063.pdf: REMATE JUDICIAL ELECTRÓNICO N° 14063
CONVOCATORIA DE REMATE
CONVOCATORIA A REMATE PÚBLICO.- En los seguidos por CREDICORP CAPITAL SOCIEDAD TITULIZADORA S.A.,
BANCO DE CREDITO DEL PERU, contra LUIS ALBERTO LOZANO REYES, INVERSIONES FORESTALES G.L.A SOCIEDAD
ANONIMA CERRADA- INFOGLA S.A.C., CLAUDIA ARAUJO ZUÑIGA, sobre proceso de EJECUCION DE GARANTIAS, en el
Expediente Judicial N° 00778-2011-0-2701-JM-CI-01, tramitado ante el JUZGADO CIVIL - SEDE TAMBOPATA DE MADRE DE
DIOS, a cargo del Juez DR(A). CORNEJO VALDERRAMA ANGELA JOANNA y del Especialista Legal DR(A). GREGORIO
TINTA CONDORI (JC), conforme a lo dispuesto en la resolución N° 42, que ordena se proceda al remate judicial electrónico a
través del aplicativo informático REM@JU, conforme a las disposiciones de la Ley N° 30229.
DATOS DEL BIEN OBJETO DE REMATE
EL CIEN POR CIENTO (100 %) DE LAS ACCIONES Y DERECHOS SOBRE EL BIEN INMUEBLE UBICADO EN PREDIO
URBANO UBICADO EN JR. SISA MZ. D LOTE 01 AAHH BARRIO NUEVO, PROVINCIA DE TAMBOPATA, DEPARTAMENTO
DE MADRE DE DIOS, INSCRITO EN LA PARTIDA ELECTRÓNICA N° 05006921 DE LA ZONA REGISTRAL N° X-SEDE
CUSCO DISTRITO DE TAMBOPATA, PROVINCIA DE TAMBOPATA, DEPARTAMENTO DE MADRE DE DIOS, inscrito en la
Partida Registral N° 05006921 del Registro de Propiedad Inmueble de MADRE DE DIOS.
CARGAS Y GRAVÁMENES
HIPOTECA: CONSTITUIDO A FAVOR DE CREDICORP CAPITAL SOCIEDAD TITULIZADORA S.A. A FIN DE GARANTIZAR
LAS OBLIGACIONES SEGÚN ASIENTO D00003, QUE CONSTITUYE EMBARGO EN FORMA DE INSCRIPCIÓN HASTA POR
LA SUMA DE $28,037.93 DÓLARES AMERICANOS
TASACIÓN
S/. 62,643.00 (SESENTA Y DOS MIL SEISCIENTOS CUARENTA Y TRES Soles)
BASE DEL REMATE EN PRIMERA CONVOCATORIA
S/. 41,762.00 (CUARENTA Y UN MIL SETECIENTOS SESENTA Y DOS Soles)
BASE DEL REMATE EN SEGUNDA CONVOCATORIA
S/. 35,497.70 (TREINTA Y CINCO MIL CUATROCIENTOS NOVENTA Y SIETE CON 70/100 Soles)
BASE DEL REMATE EN TERCERA CONVOCATORIA
S/. 30,173.05 (TREINTA MIL CIENTO SETENTA Y TRES CON 5/100 Soles)
CRONOGRAMA DEL REMATE
De conformidad con la Ley No. 30229, el cronograma del remate en primera convocatoria se genera automáticamente por el
aplicativo informático REM@JU, al que se puede acceder en el portal institucional www.pj.gob.pe
Si el remate en primera convocatoria finaliza DESIERTO, el sistema REM@JU programará de forma automática la segunda
convocatoria, generando el cronograma respectivo. Si en esta también finaliza DESIERTO, igualmente programará de forma
automática la tercera convocatoria y el cronograma correspondiente.
REPRESENTANTES
Oblarán el 10% de la tasación a través de Depósito Judicial que será pagado en el Banco de la Nación y se registrará en el
aplicativo informático REM@JU.
Asimismo deberán efectuar el pago y registro del Arancel respectivo por concepto de "Derecho a Participación en Remate".
Fecha: 10 de mayo de 2024
"""

# Para "En los seguidos por XXXXXXXXX, contra"
pattern1 = r"En los seguidos por (.*?),"
matches1 = re.search(pattern1, text)
if matches1:
    print("Seguidos por:", matches1.group(1))

# Para "Expediente Judicial XXXXXXXXX tramitado ante el"
pattern2 = r"Expediente Judicial N° (.*?),"
matches2 = re.search(pattern2, text)
if matches2:
    print("Número de Expediente:", matches2.group(1))

# Para "tramitado ante el XXXXXXXXX a cargo"
pattern3 = r"tramitado ante el (.*?),"
matches3 = re.search(pattern3, text)
if matches3:
    print("Juzgado:", matches3.group(1))

Seguidos por: CREDICORP CAPITAL SOCIEDAD TITULIZADORA S.A.
Número de Expediente: 00778-2011-0-2701-JM-CI-01
